In [31]:
import os
import librosa
import numpy as np
import pandas as pd
import catboost

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool

from xgboost import XGBClassifier
import xgboost as xgb

In [32]:
# Read csv file
#path = "csv\\mldata_v2.csv"
path = "csv\\mldata.csv"
df = pd.read_csv(path)
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav_1,66150,0.335555,0.090997,0.130189,0.003559,1773.358004,169450.829707,1972.334258,117272.640573,...,39.547077,-3.230046,36.606857,0.696385,37.766136,-5.035945,33.668550,-0.239585,43.818886,blues
1,blues.00000.wav_2,66150,0.343523,0.086782,0.112119,0.001491,1817.244034,90766.297514,2010.751494,65940.666037,...,64.819780,-6.025473,40.548813,0.127131,51.048943,-2.808956,97.221504,5.771881,60.360348,blues
2,blues.00000.wav_3,66150,0.347746,0.092495,0.130895,0.004552,1790.722358,110071.206762,2088.184750,73391.498088,...,68.306790,-1.714475,28.136944,2.329553,47.211426,-1.925621,52.922430,2.466996,33.163998,blues
3,blues.00000.wav_4,66150,0.363863,0.087207,0.131349,0.002338,1660.545231,109496.936309,1967.920582,79805.901501,...,48.543200,-3.786986,28.419546,1.153315,35.682700,-3.501979,50.610344,3.580636,32.325880,blues
4,blues.00000.wav_5,66150,0.335481,0.088482,0.142370,0.001734,1634.465076,77425.419156,1954.633566,57359.695597,...,30.829544,0.635798,44.645557,1.591107,51.415867,-3.364908,26.421090,0.501504,29.109533,blues


In [33]:
df.columns

Index(['filename', 'length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean',
       'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var',
  

In [34]:
df.shape[1]

60

## Preprocessing for machine learning

In [35]:
# Define features set. Dropping filename and label
X = df.copy()
X.drop(["filename", "length", "label"], axis=1, inplace=True)
#Converting X to numpy array as a part of preprocessing
X.to_numpy()

array([[ 3.3555517e-01,  9.0996690e-02,  1.3018920e-01, ...,
         3.3668550e+01, -2.3958503e-01,  4.3818886e+01],
       [ 3.4352323e-01,  8.6782290e-02,  1.1211886e-01, ...,
         9.7221504e+01,  5.7718810e+00,  6.0360348e+01],
       [ 3.4774640e-01,  9.2494520e-02,  1.3089524e-01, ...,
         5.2922430e+01,  2.4669962e+00,  3.3163998e+01],
       ...,
       [ 3.4781490e-01,  8.9140065e-02,  5.1905760e-02, ...,
         4.7727330e+01, -2.1486863e-01,  4.1377610e+01],
       [ 3.8680053e-01,  8.4649250e-02,  6.5967320e-02, ...,
         2.4930906e+01,  6.6620857e-01,  1.2577226e+01],
       [ 3.6872125e-01,  8.6581630e-02,  5.0242780e-02, ...,
         3.8876560e+01, -3.4117005e+00,  3.1680986e+01]])

In [36]:
# Define target vector "label"
y = df["label"].to_numpy()
y[:5]

array(['blues', 'blues', 'blues', 'blues', 'blues'], dtype=object)

In [37]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [38]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [39]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [40]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Implementing Random Forest

In [41]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=42)

In [42]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [43]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [44]:
# Calculate confusion matrix
cm = confusion_matrix(y_test, predictions)

# Use unique class labels for the confusion matrix
class_labels = sorted(set(y_test)) 
cm_df = pd.DataFrame(cm, index=[f"{label}" for label in class_labels], 
                         columns=[f"Predicted {label}" for label in class_labels])

# Confusion matrix
print("Confusion Matrix for Random Forest:")
cm_df.head()

Confusion Matrix for Random Forest:


,Predicted blues,Predicted classical,Predicted country,Predicted disco,Predicted hiphop,Predicted jazz,Predicted metal,Predicted pop,Predicted reggae,Predicted rock
blues,219,2,7,5,0,4,8,0,5,0
classical,0,238,3,0,0,8,0,0,0,1
country,10,3,210,1,0,7,2,4,10,2
disco,1,0,9,212,5,2,5,1,6,9
hiphop,0,2,2,4,212,2,2,8,18,0


In [45]:
#Evaluate the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       blues       0.90      0.88      0.89       250
   classical       0.89      0.95      0.92       250
     country       0.78      0.84      0.81       249
       disco       0.86      0.85      0.85       250
      hiphop       0.96      0.85      0.90       250
        jazz       0.84      0.88      0.86       247
       metal       0.87      0.95      0.91       250
         pop       0.92      0.87      0.89       250
      reggae       0.79      0.88      0.83       250
        rock       0.87      0.72      0.79       250

    accuracy                           0.87      2496
   macro avg       0.87      0.87      0.87      2496
weighted avg       0.87      0.87      0.87      2496



## Trying XGBoost

In [46]:
#X_train, X_test, y_train, y_test are already preprocessed and are numpy
#Labels of music genres should be converted to integers for this algorithm

#Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [47]:
# Train the XGBoost model
XGB_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
XGB_model.fit(X_train, y_train_encoded)

# Make predictions
predictions = XGB_model.predict(X_test)

# Decode the predictions back to original labels
predict_decoded = label_encoder.inverse_transform(predictions)

# Evaluate the model
print(classification_report(y_test, predict_decoded))

              precision    recall  f1-score   support

       blues       0.92      0.88      0.90       250
   classical       0.92      0.96      0.94       250
     country       0.83      0.88      0.85       249
       disco       0.90      0.90      0.90       250
      hiphop       0.93      0.90      0.91       250
        jazz       0.89      0.88      0.88       247
       metal       0.92      0.95      0.93       250
         pop       0.93      0.90      0.91       250
      reggae       0.87      0.89      0.88       250
        rock       0.88      0.82      0.85       250

    accuracy                           0.90      2496
   macro avg       0.90      0.90      0.90      2496
weighted avg       0.90      0.90      0.90      2496



In [48]:
# Calculate confusion matrix
cm = confusion_matrix(y_test, predict_decoded)

# Use unique class labels for the confusion matrix
class_labels = sorted(set(y_test)) 
cm_df = pd.DataFrame(cm, index=[f"{label}" for label in class_labels], 
                         columns=[f"Predicted {label}" for label in class_labels])

# Confusion matrix
print("Confusion Matrix for XGBoost:")
cm_df.head()

Confusion Matrix for XGBoost:


,Predicted blues,Predicted classical,Predicted country,Predicted disco,Predicted hiphop,Predicted jazz,Predicted metal,Predicted pop,Predicted reggae,Predicted rock
blues,220,1,11,2,0,2,6,0,4,4
classical,0,240,2,0,0,7,0,0,0,1
country,9,2,220,1,1,7,0,2,5,2
disco,1,1,4,225,7,1,3,3,1,4
hiphop,0,0,2,1,224,3,2,5,13,0


## Multinomial Logistical Regresstion 

In [49]:
#Encode y labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [50]:
#Split the data, stratify y_encoded for balanced splitting
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [51]:
#Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [52]:
#Using multinomial logistical regression
log_reg_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

log_reg_model.fit(X_train_scaled, y_train)


c:\Users\masce\anaconda3\envs\dev\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [53]:
#Predict on test data
prediction = log_reg_model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, prediction)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.72


In [54]:
#Classification report
print(classification_report(y_test, prediction, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

       blues       0.70      0.71      0.70       200
   classical       0.89      0.94      0.91       200
     country       0.62      0.58      0.60       199
       disco       0.66      0.67      0.66       200
      hiphop       0.80      0.64      0.71       200
        jazz       0.76      0.81      0.79       198
       metal       0.77      0.88      0.82       200
         pop       0.79      0.79      0.79       200
      reggae       0.67      0.68      0.67       200
        rock       0.51      0.48      0.50       200

    accuracy                           0.72      1997
   macro avg       0.72      0.72      0.72      1997
weighted avg       0.72      0.72      0.72      1997



## CatBoost

In [55]:
#Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [56]:
# Train CatBoost Model
catboost_model = CatBoostClassifier(
    iterations=1000,       
    learning_rate=0.1,    
    depth=6,                  
    loss_function='MultiClass',
    random_seed=42,     
    verbose=50    
)

catboost_model.fit(X_train, y_train)

0:	learn: 2.1527613	total: 313ms	remaining: 5m 13s
50:	learn: 0.8380646	total: 5.37s	remaining: 1m 39s
100:	learn: 0.5877017	total: 10.5s	remaining: 1m 33s
150:	learn: 0.4557673	total: 15.3s	remaining: 1m 25s
200:	learn: 0.3746573	total: 20.1s	remaining: 1m 19s
250:	learn: 0.3119178	total: 24.9s	remaining: 1m 14s
300:	learn: 0.2700014	total: 29.7s	remaining: 1m 9s
350:	learn: 0.2349929	total: 34.4s	remaining: 1m 3s
400:	learn: 0.2055422	total: 39.4s	remaining: 58.9s
450:	learn: 0.1823140	total: 44.3s	remaining: 53.9s
500:	learn: 0.1624248	total: 49s	remaining: 48.8s
550:	learn: 0.1459093	total: 53.7s	remaining: 43.7s
600:	learn: 0.1319315	total: 58.4s	remaining: 38.8s
650:	learn: 0.1199047	total: 1m 3s	remaining: 33.9s
700:	learn: 0.1096122	total: 1m 8s	remaining: 29.1s
750:	learn: 0.1000596	total: 1m 12s	remaining: 24.2s
800:	learn: 0.0919235	total: 1m 17s	remaining: 19.3s
850:	learn: 0.0844253	total: 1m 22s	remaining: 14.5s
900:	learn: 0.0775437	total: 1m 27s	remaining: 9.6s
950:	lea

In [57]:
# Evaluate the Model
y_pred = catboost_model.predict(X_test)
print("Accuracy on Test Set:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy on Test Set: 0.9173760640961443
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92       208
           1       0.94      0.98      0.96       204
           2       0.88      0.91      0.89       186
           3       0.88      0.90      0.89       195
           4       0.96      0.91      0.93       236
           5       0.91      0.90      0.91       189
           6       0.94      0.96      0.95       210
           7       0.94      0.91      0.92       197
           8       0.93      0.92      0.92       199
           9       0.88      0.84      0.86       173

    accuracy                           0.92      1997
   macro avg       0.92      0.92      0.92      1997
weighted avg       0.92      0.92      0.92      1997



## Testing of audio files with librosa library was made with Chat GPT assistance

In [73]:
#Test audio file
def extract_features(file_path):
    # Load the audio file
    y, sr = librosa.load(file_path, duration=30)

    # Extract features in the same order as in training
    chroma_stft_mean = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    chroma_stft_var = np.var(librosa.feature.chroma_stft(y=y, sr=sr))

    rms_mean = np.mean(librosa.feature.rms(y=y))
    rms_var = np.var(librosa.feature.rms(y=y))

    spectral_centroid_mean = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spectral_centroid_var = np.var(librosa.feature.spectral_centroid(y=y, sr=sr))

    spectral_bandwidth_mean = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    spectral_bandwidth_var = np.var(librosa.feature.spectral_bandwidth(y=y, sr=sr))

    rolloff_mean = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    rolloff_var = np.var(librosa.feature.spectral_rolloff(y=y, sr=sr))

    zero_crossing_rate_mean = np.mean(librosa.feature.zero_crossing_rate(y=y))
    zero_crossing_rate_var = np.var(librosa.feature.zero_crossing_rate(y=y))

    harmony = librosa.effects.harmonic(y)
    harmony_mean = np.mean(harmony)
    harmony_var = np.var(harmony)

    perceptr = librosa.effects.percussive(y)
    perceptr_mean = np.mean(perceptr)
    perceptr_var = np.var(perceptr)

    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    mfccs_mean = np.mean(mfccs, axis=1)
    mfccs_var = np.var(mfccs, axis=1)

    # Combine all features into a single feature vector
    features = np.hstack((
        chroma_stft_mean, chroma_stft_var,
        rms_mean, rms_var,
        spectral_centroid_mean, spectral_centroid_var,
        spectral_bandwidth_mean, spectral_bandwidth_var,
        rolloff_mean, rolloff_var,
        zero_crossing_rate_mean, zero_crossing_rate_var,
        harmony_mean, harmony_var,
        perceptr_mean, perceptr_var,
        tempo,
        mfccs_mean, mfccs_var
    ))

    # Print features for debugging
    print("Extracted Features:", features)

    return features


# Test the function

#file_path = "Resources/GTZAN/Music_files/disco.00005.wav"
#file_path = "Resources/GTZAN/Music_files/rock.00005.wav"
#file_path = "Resources/GTZAN/Music_files/classical.00005.wav"
#file_path = "Resources/GTZAN/Music_files/pop.00005.wav"
file_path = "Resources/GTZAN/Music_files/hiphop.00005.wav"
test_features = extract_features(file_path)

Extracted Features: [ 4.90667313e-01  8.14223811e-02  2.61673719e-01  5.38808526e-03
  2.70896137e+03  2.48191459e+05  2.51090241e+03  4.85303238e+04
  5.79510662e+03  4.93283873e+05  1.24614137e-01  2.61332564e-03
 -2.85360758e-04  3.25349942e-02 -2.89703161e-03  1.88680068e-02
  7.17773438e+01 -1.16755075e+01  8.19932480e+01 -8.50569725e+00
  5.27678261e+01 -1.47042198e+01  3.19647350e+01 -1.96899986e+01
  2.28694267e+01 -1.47106476e+01  1.45188112e+01 -1.00830784e+01
  1.40050716e+01 -8.07301331e+00  7.43421364e+00 -5.49781513e+00
  1.28421764e+01 -3.69733596e+00  6.21795845e+00 -2.59025145e+00
  1.05407983e-01  1.34911279e+03  3.85396332e+02  2.38736801e+02
  2.32358841e+02  1.53807510e+02  1.05960838e+02  1.15261322e+02
  8.82033997e+01  6.16255455e+01  8.11273117e+01  7.67177582e+01
  6.65447159e+01  5.20571442e+01  4.34075356e+01  3.93735504e+01
  4.33456497e+01  4.18900490e+01  4.50959702e+01  3.82146568e+01
  4.12258415e+01]


In [74]:
#Testing audio file on Random Forest

# Step 2: Check the number of features
if len(test_features) < rf_model.n_features_in_:
    # Add a placeholder for the missing feature
    missing_features = rf_model.n_features_in_ - len(test_features)
    test_features = np.append(test_features, [0] * missing_features)
elif len(test_features) > rf_model.n_features_in_:
    # Trim extra features
    test_features = test_features[:rf_model.n_features_in_]

# Step 3: Reshape for prediction
test_features = test_features.reshape(1, -1)

# Step 4: Make prediction
prediction = rf_model.predict(test_features)

print(f"Predicted genre: {prediction[0]}")

Predicted genre: hiphop
